# Discretización

## Discretizando usando umbral

A veces no nos interesa mostrar si un valor numérico es suficientemente alto o no.
Vienen bien para algunos clasificadores, como el *Bernoulli Restricted Boltzmann Machine*, y son muy populares en procesamiento de texto.

Por ejemplo, a partir del número de cigarros al día identificar si es un fumador habitual.

In [40]:
from sklearn.preprocessing import Binarizer
cigarros_semana = [5, 10, 20, 4, 8]
bin = Binarizer(threshold=8)
fumador_habitual = bin.fit_transform([cigarros_semana])
print(fumador_habitual)

[[0 1 1 0 0]]


## Discretización usando rangos

A menudo no nos interesa un valor numéricos (ej: `age`) sino convertirlo en un conjunto discreto de valores (*joven*, *adulto*, *mayor*).

La clase `K-bins` permite discretizar.

In [41]:
from sklearn.preprocessing import KBinsDiscretizer
import pandas as pd
import numpy as np
from sklearn import datasets
iris_dataset = datasets.load_iris(as_frame=True)
X_iris = iris_dataset.data.copy()
iris_targets = iris_dataset.target.copy()
#Build a discretizer object indicating three bins for every feature
est = KBinsDiscretizer(n_bins=[3, 3, 3, 3], encode='ordinal').fit(X_iris)
#Check feature maximum and minimum values 
# print(np.max(X_iris, axis = 0))
# print(np.min(X_iris, axis = 0))
#Check binning intervals
print(est.bin_edges_)

[array([4.3, 5.4, 6.3, 7.9]) array([2. , 2.9, 3.2, 4.4])
 array([1.        , 2.63333333, 4.9       , 6.9       ])
 array([0.1       , 0.86666667, 1.6       , 2.5       ])]


---

In [42]:
#Print discretization results
print(X_iris.iloc[:5,])
discretized_X = pd.DataFrame(est.transform(X_iris), columns=X_iris.columns)
print(discretized_X.iloc[:5,])

   sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)
0                5.1               3.5                1.4               0.2
1                4.9               3.0                1.4               0.2
2                4.7               3.2                1.3               0.2
3                4.6               3.1                1.5               0.2
4                5.0               3.6                1.4               0.2
   sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)
0                0.0               2.0                0.0               0.0
1                0.0               1.0                0.0               0.0
2                0.0               2.0                0.0               0.0
3                0.0               1.0                0.0               0.0
4                0.0               2.0                0.0               0.0


## Distintas estrategias de discretización

El criterio de discretización puede ser cambiado con el parámetro `strategy`.

Vamos a usar un conjunto pequeño de ejemplo:

```python
data_train_df = pd.DataFrame({'age': [30, 41, 42, 21],
                         'pelo': targets_train,
                         'ojos': ['azules', 'verdes', 'marrones', 'marrones']})
data_test_df = pd.DataFrame({'age': [25, 23],
                             'pelo': targets_test,
                             'ojos': ['verdes', 'azules']})
print(data_train_df)
```

Una tendencia común sería una discretización de ancho uniforme:


In [43]:
targets_train = ["rubio", "moreno", "pelirrojo", "azul"]
targets_test = ["moreno", "pelirrojo"]

data_train_df = pd.DataFrame({'age': [30, 41, 42, 21],
                         'pelo': targets_train,
                         'ojos': ['azules', 'verdes', 'marrones', 'marrones']})
data_test_df = pd.DataFrame({'age': [25, 23],
                             'pelo': targets_test,
                             'ojos': ['verdes', 'azules']})
print(data_train_df)

   age       pelo      ojos
0   30      rubio    azules
1   41     moreno    verdes
2   42  pelirrojo  marrones
3   21       azul  marrones


In [44]:
est = KBinsDiscretizer(n_bins=5, encode='ordinal', strategy='uniform')
age_disc = est.fit_transform(data_train_df[['age']])
print(est.bin_edges_)
print(age_disc)

[array([21. , 25.2, 29.4, 33.6, 37.8, 42. ])]
[[2.]
 [4.]
 [4.]
 [0.]]


. . .

No todos los rangos tienen interés, pueden concentrarse.

---

A menudo la mejor estrategia depende de la frecuencia, procurando que todos los intervalos discretizados tengan la misma cantidad de ejemplos (comportamiento por defecto).

In [45]:
est = KBinsDiscretizer(n_bins=5, encode='ordinal', strategy='quantile')
age_disc = est.fit_transform(data_train_df[['age']])
print(est.bin_edges_)
print(age_disc)

[array([21. , 26.4, 32.2, 38.8, 41.4, 42. ])]
[[1.]
 [3.]
 [4.]
 [0.]]


De esta manera, discretiza más en detalle los intervalos más comunes.

---

La otra opción es la estrategia `kmean` que aplica una clasificación `kmeans` sobre cada atributo.

In [46]:
#Build a discretizer object indicating three bins for every feature and using the kmeans strategy
est = KBinsDiscretizer(n_bins=[3, 3, 3, 3], encode='ordinal', strategy='kmeans').fit(X_iris)
#Check binning intervals and results
print(est.bin_edges_)
discretized_X = pd.DataFrame(est.transform(X_iris), columns=X_iris.columns)
print(discretized_X.iloc[:5,])

[array([4.3       , 5.53309253, 6.54877049, 7.9       ])
 array([2.        , 2.85216858, 3.43561538, 4.4       ])
 array([1.        , 2.87637037, 4.95950081, 6.9       ])
 array([0.1       , 0.79151852, 1.70547504, 2.5       ])]
   sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)
0                0.0               2.0                0.0               0.0
1                0.0               1.0                0.0               0.0
2                0.0               1.0                0.0               0.0
3                0.0               1.0                0.0               0.0
4                0.0               2.0                0.0               0.0


c:\Users\julian\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
c:\Users\julian\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
c:\Users\julian\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
c:\Users\julian\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Wi

## Discretización con MDLP de Fayyad

This is an implementation of Usama Fayyad's entropy based expert binning method.

Please read the original (http://web.donga.ac.kr/kjunwoo/files/Multi%20interval%20discretization%20of%20continuous%20valued%20attributes%20for%20classification%20learning.pdf) here for more information.

In [47]:
from MDLP import MDLP_Discretizer

feature_names, class_names = iris_dataset['feature_names'], iris_dataset['target_names']

numeric_features = np.arange(X_iris.shape[1]) 
discretizer = MDLP_Discretizer(features=numeric_features)

#fit necesita numpy arrays
X = X_iris.to_numpy()
y = iris_targets.to_numpy()

discretizer.fit(X, y)

X_mdlp = discretizer.transform(X) # recordar que habría que aplicar este mismo transform a los datos de test si se hace una partición

# Imprimimos el principio del conjunto original y el transformado
print(X[:5,])
print(X_mdlp[:5,])

# Ver los puntos de corte del primer atributo, por ejemplo
print(discretizer._cuts[0])
print(discretizer._bin_descriptions)

[[5.1 3.5 1.4 0.2]
 [4.9 3.  1.4 0.2]
 [4.7 3.2 1.3 0.2]
 [4.6 3.1 1.5 0.2]
 [5.  3.6 1.4 0.2]]
[[0. 2. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 2. 0. 0.]]
[5.55, 6.15]
{0: {0: '-inf_to_5.55', 1: '5.55_to_6.15', 2: '6.15_to_inf'}, 1: {0: '-inf_to_2.95', 1: '2.95_to_3.3499999999999996', 2: '3.3499999999999996_to_inf'}, 2: {0: '-inf_to_2.45', 1: '2.45_to_4.75', 2: '4.75_to_inf'}, 3: {0: '-inf_to_0.8', 1: '0.8_to_1.75', 2: '1.75_to_inf'}}


## Discretización usando CAIM

CAIM es un algoritmo de discretización muy usado. En Python estaba disponible en el paquete `caimcaim`, pero por problema de dependencias lo he incluido en el zip:

---

In [48]:
from caimcaim import CAIMD
caim_dis = CAIMD()
caim_dis.fit(X_iris, iris_targets)
print(X_iris.iloc[:5,])
discretized_X = caim_dis.transform(X_iris)
print(discretized_X.iloc[:5,:])

Categorical []
# 0  GLOBAL CAIM  26.636271740334553
# 1  GLOBAL CAIM  17.382507167267576
# 2  GLOBAL CAIM  45.55892255892255
# 3  GLOBAL CAIM  46.16156736446592
   sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)
0                5.1               3.5                1.4               0.2
1                4.9               3.0                1.4               0.2
2                4.7               3.2                1.3               0.2
3                4.6               3.1                1.5               0.2
4                5.0               3.6                1.4               0.2
   sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)
0                0.0               2.0                0.0               0.0
1                0.0               2.0                0.0               0.0
2                0.0               2.0                0.0               0.0
3                0.0               2.0                0.0               0.0
4  

## Ejercicios de discretización

Aparte de considerar nuestro querido conjunto *Iris*, planteamos practicar sobre el conjunto  "IMDB-Movie-Data.csv".

In [49]:
import pandas as pd
df = pd.read_csv("IMDB-Movie-Data.csv")
df.columns

Index(['Rank', 'Title', 'Genre', 'Description', 'Director', 'Actors', 'Year',
       'Runtime (Minutes)', 'Rating', 'Votes', 'Revenue (Millions)',
       'Metascore'],
      dtype='object')

1. Discretiza los ingresos en 10 intervalos de igual rango.

2. Discretiza los ingresos en 10 intervalos según la frecuencia.

3. Discretizar las películas entre populares (un 8 o más) y menos.

4. Aplica MDLP y CAIM sobre el conjunto elegido.

5. Usa un decision tree sobre diferentes estrategias de discretización. ¿Qué diferencias en la profundidad del árbol y el rendimiento observas?